In [1]:
from __future__ import print_function, division
from pysis.util import file_variations
import os
from pysis import CubeFile
from pysis.isis import cubenorm, handmos, hi2isis, hical, histitch, spiceinit, getkey
import subprocess as sp
from pysis.exceptions import ProcessError
import sys

# case 2: turn IMG to cub, give information of it (spiceinit), don't calibrate it
def nocal_hi(img_name):
    (img_name, cub_name) = file_variations(img_name, ['.IMG', '.cub'])
    # something wrong here, if just put one extension in it, the name will contain brankets
    try:
        hi2isis(from_=img_name, to=cub_name)
        spiceinit(from_=cub_name)
    except ProcessError as e:
        print(e.stdout)
        print(e.stderr)
        sys.exit()

# stitch _0 and _1 together and normalize it
def stit_norm_hi(img1, img2):
    cub = img1[0:20]+'.cub'
    (cub, norm) = file_variations(cub, ['.cub', '.norm.cub'])   # same here
    try:
        histitch(from1=img1, from2=img2, to=cub)
        cubenorm(from_=cub, to=norm)
    except ProcessError as e:
        print(e.stdout)
        print(e.stderr)
        sys.exit()
        
def hi2mos(nm):
    os.chdir('/Users/klay6683/data/planet4/season2_3_EDRs')
    print('Processing the EDR data associated with '+nm)

    mos_name = 'redMosaic'+nm+'.cub'
#     status = os.path.isfile(mos_name)
    status = False
    if status is True:
        print('skip processing '+nm+'...')
        return nm, False
    else:
        nm = nm+'_RED'
        channel = [4, 5]
        ccd = [0, 1]

        for c in channel:
            for chip in ccd:
                nocal_hi(nm+str(c)+'_'+str(chip)+'.IMG')

            stit_norm_hi(nm+str(c)+'_0.cub', nm+str(c)+'_1.cub')

        # handmos part
        im0 = CubeFile.open(nm+'4.norm.cub')  # use CubeFile to get lines and samples
        # use linux commands to get binning mode
        bin = int(getkey(from_=nm+'4.norm.cub', objname="isiscube", grpname="instrument",
                  keyword="summing"))

        # because there is a gap btw RED4 & 5, nsamples need to first make space
        # for 2 cubs then cut some overlap pixels
        try:
            handmos(from_=nm+'4.norm.cub', mosaic=mos_name, nbands=1, outline=1, outband=1,
                    create='Y', outsample=1, nsamples=im0.samples*2-48//bin, 
                    nlines=im0.lines)
        except ProcessError as e:
            print("STDOUT:", e.stdout)
            print("STDERR:", e.stderr)
        im0 = CubeFile.open(nm+'5.norm.cub')  # use CubeFile to get lines and samples

        # deal with the overlap gap between RED4 & 5:
        handmos(from_=nm+'5.norm.cub', mosaic=mos_name, outline=1, outband=1, create='N',
                outsample=im0.samples-48//bin+1)  
        return nm, True    


def cleanup(data_dir, img):
    # do some cleanup removing temporary files
    # removing ISIS cubes made during processing that aren't needed
    fs = glob.glob(data_dir+'/'+img+'_RED*.cub')

    print(fs)
    for f in fs:
        os.remove(f)

    # removing the normalized files

    fs = glob.glob(data_dir+'/'+img+'_RED*.norm.cub')
    for f in fs:
        os.remove(f)

    # remove the raw EDR data

    fs = glob.glob(data_dir+'/'+img+'_RED*.IMG')
    for f in fs:
        os.remove(f)



In [2]:
img_names = pd.read_table('/Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt',
                          header=None, squeeze=True)
img_names.head()

0    ESP_013113_0985
1    ESP_013034_0985
2    ESP_012889_0985
3    ESP_012744_0985
4    ESP_012691_0985
Name: 0, dtype: object

In [3]:
cd ~/data/planet4/season2_3_EDRs/

/Users/klay6683/data/planet4/season2_3_EDRs


In [4]:
from ipyparallel import Client
c = Client()

In [5]:
lbview = c.load_balanced_view()
dview = c.direct_view()

In [6]:
%%px 
from __future__ import print_function,division
from pysis.util import file_variations
import os
from pysis import CubeFile
from pysis.isis import cubenorm, handmos, hi2isis, hical, histitch, spiceinit, getkey
import subprocess as sp
from pysis.exceptions import ProcessError
import sys

In [7]:
dview.push({'nocal_hi':nocal_hi,
            'stit_norm_hi':stit_norm_hi})

<AsyncResult: finished>

In [12]:
results = lbview.map_async(hi2mos, img_names)

In [13]:
from iuvs.multitools import nb_progress_display

In [14]:
nb_progress_display(results, img_names)

In [15]:
for res in results:
    print(res)

('ESP_013113_0985_RED', True)
('ESP_013034_0985_RED', True)
('ESP_012889_0985_RED', True)
('ESP_012744_0985_RED', True)
('ESP_012691_0985_RED', True)
('ESP_012467_0985_RED', True)
('ESP_012322_0985_RED', True)
('ESP_012256_0985_RED', True)
('ESP_011900_0985_RED', True)
('ESP_011729_0985_RED', True)
('ESP_011702_0985_RED', True)
('ESP_011623_0985_RED', True)
('ESP_011557_0985_RED', True)
('ESP_011544_0985_RED', True)
('ESP_011491_0985_RED', True)
('ESP_022699_0985_RED', True)
('ESP_021460_0985_RED', True)
('ESP_020959_0985_RED', True)
('ESP_020748_0985_RED', True)
('ESP_020194_0985_RED', True)
('ESP_020128_0985_RED', True)
('ESP_020049_0985_RED', True)
('ESP_021829_0985_RED', True)
('ESP_021684_0985_RED', True)
('ESP_021671_0985_RED', True)
('ESP_021605_0985_RED', True)
('ESP_021526_0985_RED', True)
('ESP_020827_0985_RED', True)
('ESP_020339_0985_RED', True)
('ESP_020115_0985_RED', True)


## cleanup

In [17]:
!rm -f *RED*.cub

# xy2latlon

In [18]:
def xy2latlon(p):
    pass

In [24]:
df = pd.read_csv('/Users/klay6683/Dropbox/data/planet4/season23_catalog/ESP_021684_0985_blotches.csv',
            index_col=0)
df.head()

,image_x,image_y,angle,radius_1,radius_2,x,y
0,1054.102563,69636.461538,20.386018,24.273165,20.581054,1054.102563,69636.461538
1,782.143410,69245.255814,17.057963,21.469380,17.194975,782.143410,69245.255814
2,808.658729,69585.880952,15.856553,24.477685,19.156379,808.658729,69585.880952
3,2778.413207,47754.796226,7.092199,11.000606,10.640708,2778.413207,47754.796226
4,2676.128571,47665.000000,3.860177,11.028818,10.452894,2676.128571,47665.000000


In [49]:
from pysis.isis import campt
from pysis.exceptions import ProcessError
import pvl

In [43]:
from pathlib import Path
edrpath = Path('/Users/klay6683/data/planet4/season2_3_EDRs/')
clusterpath = Path('/Users/klay6683/Dropbox/data/planet4/season23_catalog/')

In [135]:
fnotches = list(clusterpath.glob('*_fnotches*'))

In [136]:
fnotches[0]

PosixPath('/Users/klay6683/Dropbox/data/planet4/season23_catalog/ESP_011296_0975_fnotches.csv')

In [137]:
pd.read_csv(fnotches[0], index_col=0)

,fan_image_x,fan_image_y,fan_angle,fan_spread,fan_distance,fan_x,fan_y,blotch_image_x,blotch_image_y,blotch_angle,blotch_radius_1,blotch_radius_2,blotch_x,blotch_y,fnotch_value
0,612.763510,7189.901442,-158.530541,65.184955,36.362188,612.763510,7189.901442,596.160853,7183.073644,-10.946682,19.322250,15.003017,596.160853,7183.073644,0.426667
0,572.375000,7174.000000,23.344633,23.979153,47.651901,572.375000,7174.000000,596.160853,7183.073644,-10.946682,19.322250,15.003017,596.160853,7183.073644,0.085106
0,213.605506,7165.379892,-157.443202,66.986692,40.885272,213.605506,7165.379892,196.587607,7157.106838,-13.139227,20.797405,17.382564,196.587607,7157.106838,0.493506
0,174.833333,7154.333333,11.031186,117.844975,43.825654,174.833333,7154.333333,196.587607,7157.106838,-13.139227,20.797405,17.382564,196.587607,7157.106838,0.071429
0,418.142857,6979.857143,-163.924480,60.481762,32.612338,418.142857,6979.857143,406.155303,6976.371212,-17.549192,17.757485,13.575457,406.155303,6976.371212,0.388889
0,756.411765,6581.823529,-157.370414,63.862519,31.956122,756.411765,6581.823529,745.942222,6578.008333,-3.173637,17.640700,14.012971,745.942222,6578.008333,0.361702
0,287.041667,7085.583333,-151.739362,52.675087,27.182625,287.041667,7085.583333,277.974638,7080.137681,-42.518529,13.761094,11.862814,277.974638,7080.137681,0.342857
0,116.588235,6996.764706,-153.059053,70.504503,35.796345,116.588235,6996.764706,103.856482,6991.898148,-21.419229,21.040848,16.441664,103.856482,6991.898148,0.485714
0,293.894737,6978.842105,-155.483854,68.738045,41.096732,293.894737,6978.842105,276.591270,6971.150794,-12.024738,26.062811,19.382369,276.591270,6971.150794,0.475000
0,797.604167,6666.833333,-151.483926,52.697542,44.101117,797.604167,6666.833333,784.238760,6659.773256,-8.371114,19.694798,16.116172,784.238760,6659.773256,0.358209


In [106]:
df['image_x image_y'.split()].to_csv(s, header=False, index=False)

In [107]:
try:
    _ = campt(from_=s, to='ESP_021684_0985_blotches_campt.txt',
              format='flat', append='no', coordlist='coordlist.txt',
              coordtype='image')
except ProcessError as e:
    print(e.stderr)

b'**I/O ERROR** Unable to open [<_io.StringIO object at 0x107684948>].\n'


In [130]:
obsids = !cat /Users/klay6683/Dropbox/data/planet4/season2_3_image_names.txt

In [131]:
def xy2latlon(obsid):
    from pathlib import Path
    from pysis.isis import campt
    from pysis.exceptions import ProcessError
    coords = ['image_x','image_y']
    edrpath = Path('/Users/klay6683/data/planet4/season2_3_EDRs/')
    clusterpath = Path('/Users/klay6683/Dropbox/data/planet4/season23_catalog/') 
    for marking in ['_fans', '_blotches']:
        inpath = clusterpath / (obsid + marking + '.csv')
        df = pd.read_csv(inpath, index_col=0)
        temppath = inpath.with_suffix('.tocampt')
        df[coords].to_csv(str(temppath), header=False, index=False)
        mosaicname = 'redMosaic' + obsid + '.cub'
        mosaicpath = edrpath / mosaicname
        savename = inpath.stem + '_campt_out.csv'
        savepath = edrpath / savename
        try:
            campt(from_=mosaicname, to=savepath, format='flat', append='no',
                  coordlist=temppath, coordtype='image')
        except ProcessError as e:
            print(e.stderr)
            return obsid, False
    return obsid, True

In [119]:
savepath = xy2latlon(fnames[0])

In [128]:
c = Client()

In [129]:
lbview = c.load_balanced_view()

In [132]:
results = lbview.map_async(xy2latlon, obsids)

In [133]:
nb_progress_display(results, obsids)